# Assignment 03

**Part One**

Generate Girvan-Newman (GN) networks varying:
- $z_{in}, z_{out}$ or the probabilities matrix
- communities number
- communities size/proportion

Discuss the properties of networks spectrum. Compare it with random networks without communities (ER)

**Part Two**

Compare different modularity optimization methods applied to 2 or more large networks. Discuss its advantages and disadvantages.

In [1]:
import igraph as ig
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import process_time
from numpy import linalg
import math

%config Completer.use_jedi = False

To generate a SBM network we need to define the preference matrix. Which is a $K \times K$ matrix, where $K$ is the number of groups. The probability of creating an edge between vertices from groups $i$ and $j$ is given by element $(i,j)$. In our case, $K = 2$. 

Also, $p_{0,0}$ is the probability of one member of group 0 connect to a node of group 0. This is analogue for $p_{1,1}$. $p_{0,1}$ represents the probability of a connection from a vertice from group 0 to a vertice in group 1.

$$
\begin{bmatrix}
p_{0,0}, p_{0,1} \\
p_{1, 0}, p_{1,1}
\end{bmatrix}
$$

## Part One ##
## Experiment methodology
We will create several networks and analyze their communities. We fixed the sum $z_{in} + z_{out}$ and changed the values for each $z$.

Also we varied the proportion of nodes within the communities. Each community will have 20%, 50% and 80% of the nodes. 

Finally, we repeated the experiment for 2, 3 and 4 communities.

First, let's analyze how the spectra looks like for small probabilities of connection 

In [23]:
z_max = 6
z_in_values = np.arange(1, z_max, 1)
z_out_values = [z_max - i for i in z_in_values]
community_proportion = [0.2, 0.5]
ensemble_size = 30

## Part 1.1

In [24]:
start = process_time()
idx = 4
proportion_index = 1
z_in = z_in_values[idx]
z_out = z_out_values[idx]
n = 1000
block_sizes = [int(community_proportion[proportion_index]*n), 
               # workaround to precision float issues
               int(np.round(1 - community_proportion[proportion_index], 2)*n)]
# pm = [[z_in/block_sizes[0], z_out/n],
#       [z_out/n, z_in/block_sizes[1]],
#      ]

pm = [[np.log(block_sizes[0])/block_sizes[0], 0.01],
      [0.01, np.log(block_sizes[1])/block_sizes[1]],
     ]

sbm_graphs = dict()
er_graphs = dict()

sbm_graphs["graphs"] = list()
sbm_graphs["fidler_eigenvalue"] = list()
# sbm_graphs["degree"] = list()


er_graphs["graphs"] = list()
er_graphs["fidler_eigenvalue"] = list()
# er_graphs["degree"] = list()



for i in range(ensemble_size):
    # Generate SBM graphs
    graph = ig.Graph.SBM(n = n, pref_matrix = pm, block_sizes = block_sizes, directed=False, loops=False)
    sbm_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
    
    # Generate ER graphs
    probability = math.ceil(np.mean(graph.degree()))/n
    graph = ig.Graph.Erdos_Renyi(n = n, p = probability)
    er_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))

In [25]:
spectral_analysis = pd.DataFrame({"network": [], "fidler_eigenvalue": []})
spectral_analysis["fidler_eigenvalue"] = sbm_graphs["fidler_eigenvalue"] + er_graphs["fidler_eigenvalue"]
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network_id"] = np.arange(0, ensemble_size, 1).tolist() + np.arange(0, ensemble_size, 1).tolist()

In [26]:
np.round(spectral_analysis.pivot_table(index = ["network"],
                             values = ["fidler_eigenvalue"],
                             aggfunc = ["mean", "std"], ), 2)

,mean,std
,fidler_eigenvalue,fidler_eigenvalue
network,,
er,1.98,0.68
sbm,2.10,0.54


The algebraic connectivity for 10 realizations is much higher for SBM networks, altough they have a higher variation in its values due to its higher standard deviation. SBM in average had a 2.23 algebraic connectivity against 1.55 for the ER models. If we increse the average degree in ER we should expect an increase in its algebraic connectivity. Now, we'll change the proportion of the communities

## Part 1.2

In [27]:
start = process_time()
idx = 4
proportion_index = 0
z_in = z_in_values[idx]
z_out = z_out_values[idx]
n = 1000
block_sizes = [int(community_proportion[proportion_index]*n), 
               # workaround to precision float issues
               int(np.round(1 - community_proportion[proportion_index], 2)*n)]
# pm = [[z_in/block_sizes[0], z_out/n],
#       [z_out/n, z_in/block_sizes[1]],
#      ]

pm = [[np.log(block_sizes[0])/block_sizes[0], 0.01],
      [0.01, np.log(block_sizes[1])/block_sizes[1]],
     ]

sbm_graphs = dict()
er_graphs = dict()

sbm_graphs["graphs"] = list()
sbm_graphs["fidler_eigenvalue"] = list()
# sbm_graphs["degree"] = list()


er_graphs["graphs"] = list()
er_graphs["fidler_eigenvalue"] = list()
# er_graphs["degree"] = list()



for i in range(ensemble_size):
    # Generate SBM graphs
    graph = ig.Graph.SBM(n = n, pref_matrix = pm, block_sizes = block_sizes, directed=False, loops=False)
    sbm_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
    
    # Generate ER graphs
    probability = math.ceil(np.mean(graph.degree()))/n
    graph = ig.Graph.Erdos_Renyi(n = n, p = probability)
    er_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
end = process_time()

In [28]:
spectral_analysis = pd.DataFrame({"network": [], "fidler_eigenvalue": []})
spectral_analysis["fidler_eigenvalue"] = sbm_graphs["fidler_eigenvalue"] + er_graphs["fidler_eigenvalue"]
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network_id"] = np.arange(0, ensemble_size, 1).tolist() + np.arange(0, ensemble_size, 1).tolist()

In [29]:
np.round(spectral_analysis.pivot_table(index = ["network"],
                             values = ["fidler_eigenvalue"],
                             aggfunc = ["mean", "std"], ), 2)

,mean,std
,fidler_eigenvalue,fidler_eigenvalue
network,,
er,1.26,0.45
sbm,1.11,0.59


Now, we changed the community proportion to 20% and 80%. We can see that the algbraic connectivity now is pretty close for both network. However, this time the SBM networks varies more. This is interesting because as we increase one of the community proportion, conssequently, all the rest of nodes will be assigned to the other community. And this makes the network more similar to a random network. The extreme would be a network with 100% nodes in one community and 0% in the other, or vice-versa. 

## Part 1.3

Now, we'll change the communities number. Let's create a network with 3 communities

In [30]:
start = process_time()
idx = 4
proportion_index = 0
z_in = z_in_values[idx]
z_out = z_out_values[idx]
n = 1000
community_proportion = 0.33
block_sizes = [int(community_proportion*n), int(community_proportion*n), int(np.round(1 - 2*community_proportion, 2)*n)]

pm = [[np.log(block_sizes[0])/block_sizes[0], 0.01, 0.01],
      [0.01, np.log(block_sizes[1])/block_sizes[1], 0.01],
      [0.01, 0.01, np.log(block_sizes[2])/block_sizes[2]]
     ]

sbm_graphs = dict()
er_graphs = dict()

sbm_graphs["graphs"] = list()
sbm_graphs["fidler_eigenvalue"] = list()
# sbm_graphs["degree"] = list()


er_graphs["graphs"] = list()
er_graphs["fidler_eigenvalue"] = list()
# er_graphs["degree"] = list()



for i in range(ensemble_size):
    # Generate SBM graphs
    graph = ig.Graph.SBM(n = n, pref_matrix = pm, block_sizes = block_sizes, directed=False, loops=False)
    sbm_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
    
    # Generate ER graphs
    probability = math.ceil(np.mean(graph.degree()))/n
    graph = ig.Graph.Erdos_Renyi(n = n, p = probability)
    er_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
end = process_time()

spectral_analysis = pd.DataFrame({"network": [], "fidler_eigenvalue": []})
spectral_analysis["fidler_eigenvalue"] = sbm_graphs["fidler_eigenvalue"] + er_graphs["fidler_eigenvalue"]
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network_id"] = np.arange(0, ensemble_size, 1).tolist() + np.arange(0, ensemble_size, 1).tolist()

np.round(spectral_analysis.pivot_table(index = ["network"],
                             values = ["fidler_eigenvalue"],
                             aggfunc = ["mean", "std"], ), 2)

,mean,std
,fidler_eigenvalue,fidler_eigenvalue
network,,
er,2.60,0.62
sbm,2.69,0.65


As we can see, there was an increase in algebraic connectivity of SBM networks. We can say that the ER remained the same. As the number of components increases

In [31]:
start = process_time()
idx = 4
z_in = z_in_values[idx]
z_out = z_out_values[idx]
n = 1000
community_proportion = 0.25
block_sizes = [int(community_proportion*n), int(community_proportion*n), int(community_proportion*n), int(community_proportion*n)]

pm = [[np.log(block_sizes[0])/block_sizes[0], 0.01, 0.01, 0.01],
      [0.01, np.log(block_sizes[1])/block_sizes[1], 0.01, 0.01],
      [0.01, 0.01, np.log(block_sizes[2])/block_sizes[2], 0.01],
      [0.01, 0.01, 0.01, np.log(block_sizes[3])/block_sizes[3]]
     ]

sbm_graphs = dict()
er_graphs = dict()

sbm_graphs["graphs"] = list()
sbm_graphs["fidler_eigenvalue"] = list()
# sbm_graphs["degree"] = list()


er_graphs["graphs"] = list()
er_graphs["fidler_eigenvalue"] = list()
# er_graphs["degree"] = list()



for i in range(ensemble_size):
    # Generate SBM graphs
    graph = ig.Graph.SBM(n = n, pref_matrix = pm, block_sizes = block_sizes, directed=False, loops=False)
    sbm_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
    
    # Generate ER graphs
    probability = math.ceil(np.mean(graph.degree()))/n
    graph = ig.Graph.Erdos_Renyi(n = n, p = probability)
    er_graphs["graphs"].append(graph)
    
    laplacian_matrix = ig.Graph.laplacian(graph, normalized = False)
    eigenvalues, eigenvectors = linalg.eig(laplacian_matrix)    
    fidler_eigenvector = eigenvectors[1]    
    fidler_eigenvalue = sorted(eigenvalues)[1]
    er_graphs["fidler_eigenvalue"].append(np.round(fidler_eigenvalue, 2))
end = process_time()

spectral_analysis = pd.DataFrame({"network": [], "fidler_eigenvalue": []})
spectral_analysis["fidler_eigenvalue"] = sbm_graphs["fidler_eigenvalue"] + er_graphs["fidler_eigenvalue"]
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network"] = ["sbm"] * ensemble_size + ["er"] * ensemble_size
spectral_analysis["network_id"] = np.arange(0, ensemble_size, 1).tolist() + np.arange(0, ensemble_size, 1).tolist()

np.round(spectral_analysis.pivot_table(index = ["network"],
                             values = ["fidler_eigenvalue"],
                             aggfunc = ["mean", "std"], ), 2)

,mean,std
,fidler_eigenvalue,fidler_eigenvalue
network,,
er,2.76,0.88
sbm,2.78,0.72


## Part 2

Now, we'll analyze the modularity optimization methods applied in real networks. The first network is the Wikipedia Graph.

In [32]:
finished_paths = pd.read_csv("../datasets/wikispeedia_paths-and-graph/paths_finished.tsv", 
                                delimiter = "\t", 
                                skiprows = 15, 
                                names = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
articles = pd.read_csv("../datasets/wikispeedia_paths-and-graph/articles.tsv", 
                                delimiter = "\t", 
                                skiprows = 11, 
                                names = ['article']).reset_index().rename(columns = {"index": "article_id"})
categories = pd.read_csv("../datasets/wikispeedia_paths-and-graph/categories.tsv", 
                                delimiter = "\t", 
                                skiprows = 12, 
                                names = ['article', 'category'])

In [34]:
def replace_back_page(path):
    while "<" in path:
        idx = path.index("<")
        path[idx] = path[idx - 2]
    return path
    
def parse_path(unparsed_path, delimiter = ";"):
    nodes = unparsed_path.split(delimiter)
    replaced = replace_back_page(nodes)
    return replaced

In [35]:
parsed_paths = list()
for path in finished_paths['path'].tolist():
    parsed_paths.append(parse_path(path))

In [36]:
def path_array_to_tuple(path_array):
    tuple_array = list()
    for i in range(0, len(path_array)-1, 1):
        tuple_array.append((path_array[i], path_array[i+1]))
    return tuple_array

edges = list()
for p in parsed_paths:
    edges.extend(path_array_to_tuple(p))

In [38]:
wikipedia_graph = ig.Graph()
wikipedia_graph.add_vertices(articles['article'].tolist())#, attributes = categories.set_index(keys = 'article').to_dict()['category'])

In [50]:
articles['article']

0       %C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1                             %C3%85land
2                     %C3%89douard_Manet
3                              %C3%89ire
4             %C3%93engus_I_of_the_Picts
                      ...               
4599                             Zionism
4600                           Zirconium
4601                           Zoroaster
4602                        Zuid-Gelders
4603                                Zulu
Name: article, Length: 4604, dtype: object

In [39]:
%%time

def add_edges_to_graph(graph, edges, autoconnect = False):
    for e in edges:
        # Auto-connections are not allowed
        if e[0] != e[1] and ~autoconnect:
            try:
                graph.add_edges([e])
            except:
                pass
        elif autoconnect:
            try:
                graph.add_edges([e])
            except:
                pass
add_edges_to_graph(wikipedia_graph, edges[:1000])

CPU times: user 58.4 ms, sys: 2.32 ms, total: 60.8 ms
Wall time: 62.9 ms


In [48]:
email_edges = pd.read_table("../datasets/email-EU/email-Eu-core.txt", names = ["sender_id", "receipt_id"], delimiter = " ")
department_labels = pd.read_table("../datasets/email-EU/email-Eu-core-department-labels.txt", names = ["node_id", "department_id"], delimiter = " ")

In [61]:
distinct_senders = email_edges["sender_id"].unique().tolist()
distinct_receipt = email_edges["receipt_id"].unique().tolist()
distinct_nodes = set(distinct_senders + distinct_receipt)

In [74]:
r = email_edges.to_records(index = False, )

In [75]:
emails_graph = ig.Graph()
emails_graph.add_vertices(list(distinct_nodes))#, attributes = categories.set_index(keys = 'article').to_dict()['category'])
add_edges_to_graph(emails_graph, email_edges.to_records(index = False, ))

In [ ]:
# sample_graph = er_graphs["graphs"][9]
# fig, ax = plt.subplots(figsize = (10, 6))
# ig.plot(sample_graph, 
#         target = ax,
#        vertex_color = block_sizes[0] * ["blue"] + block_sizes[1] * ["red"] )

In [ ]:
elapsed_time = np.round(end - start, 4)
print('Elapsed time:', elapsed_time)

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize = (10, 6))
_ = sns.lineplot(x = [i for i in range(df_plot.shape[0])],
               y = df_plot["eigenvalues"])
ax.set_ylabel('Fidler value')
sns.despine()

In [ ]:
spectral_analysis[spectral_analysis["fidler_value"] == 0]

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize = (10, 6))

_ = sns.scatterplot(x = spectral_analysis["node"],
                    y = spectral_analysis["fidler_value"],
                    hue = spectral_analysis["community_groundtruth"],
                    palette = {0: "red", 1: "blue"},
                   )
# _ = sns.lineplot(x = spectral_analysis["node"],
#                 y = 0, color = "green", style=True, dashes=[(2,2)]
#                 )
ax.set_ylabel('Fidler value')
sns.despine()

In [ ]:
er_graph = ig.Graph.Erdos_Renyi(n = n, p = 0.3)
er_laplacian_matrix = ig.Graph.laplacian(er_graph, )

er_eigenvalues, er_eigenvectors = linalg.eig(er_laplacian_matrix)
er_fidler_eigenvector = er_eigenvectors[1]

er_spectral_analysis = pd.DataFrame({"node": [i for i in range(n)],"eigenvalues": er_eigenvalues})

er_spectral_analysis["fidler_value"] = er_fidler_eigenvector
er_spectral_analysis["community_membership"] = er_spectral_analysis["fidler_value"].apply(lambda x: 0 if x >= 0 else 1)
# er_spectral_analysis["community_groundtruth"] = block_sizes[0] * [0] + block_sizes[1] * [1]

In [ ]:
df_plot = er_spectral_analysis.sort_values(by = ["eigenvalues"])
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize = (10, 6))
_ = sns.lineplot(x = [i for i in range(df_plot.shape[0])],
               y = df_plot["eigenvalues"])
ax.set_ylabel('Fidler value')
sns.despine()